<a href="https://colab.research.google.com/github/pren1/VAST/blob/master/Splitter_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# %tensorflow_version 1.x

In [2]:
!rm -rf VAST/
!git clone https://github.com/pren1/VAST.git
!pip3 install soundfile
!pip3 install librosa

Cloning into 'VAST'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 118 (delta 52), reused 58 (delta 21), pack-reused 0
Receiving objects: 100% (118/118), 44.20 MiB | 11.52 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [0]:
from VAST.util import *
import pdb
import pprint
from tqdm import tqdm
import numpy as np
import IPython
import soundfile as sf
import matplotlib.pyplot as plt
import librosa
import os
from tqdm import trange

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import math
import tensorflow as tf

def trunk_network(conv_size = 64, LSTM_size = 400, input_size = (51, 601)):
    inputs = tf.keras.layers.Input(input_size)
    x = tf.keras.layers.Reshape((input_size[0],  input_size[1], 1))(inputs)
    # cnn1
    x = tf.keras.layers.ZeroPadding2D(padding=((0, 0), (3, 3)))(x)
    x = tf.keras.layers.Conv2D(conv_size, (1, 7))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # cnn2
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (0, 0)))(x)
    x = tf.keras.layers.Conv2D(conv_size, (7, 1))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # # cnn3
    # x = tf.keras.layers.ZeroPadding2D(padding=2)(x)
    # x = tf.keras.layers.Conv2D(conv_size, (5, 5))(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.ReLU()(x)

    # # cnn4
    # x = tf.keras.layers.ZeroPadding2D(padding=((4, 4), (2, 2)))(x)
    # x = tf.keras.layers.Conv2D(conv_size, (5, 5), dilation_rate=(2, 1))(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.ReLU()(x)

    # # cnn5
    # x = tf.keras.layers.ZeroPadding2D(padding=((8, 8), (2, 2)))(x)     
    # x = tf.keras.layers.Conv2D(conv_size, (5, 5), dilation_rate=(4, 1))(x)       
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.ReLU()(x)

    # # cnn6
    # x = tf.keras.layers.ZeroPadding2D(padding=((16, 16), (2, 2)))(x)                 
    # x = tf.keras.layers.Conv2D(conv_size, (5, 5), dilation_rate=(8, 1))(x) 
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.ReLU()(x)

    # # cnn7 TensorShape([None, 51, 601, conv_size])
    # x = tf.keras.layers.ZeroPadding2D(padding=((32, 32), (2, 2)))(x)
    # x = tf.keras.layers.Conv2D(conv_size, (5, 5), dilation_rate=(16, 1))(x)     
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.ReLU()(x)

    # cnn8 TensorShape([None, 51, 601, 8])
    x = tf.keras.layers.Conv2D(8, (1, 1))(x)     
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    'Reshape x shape TensorShape([None, 51, 4808])'
    x = tf.keras.layers.Reshape((tf.keras.backend.int_shape(x)[1], tf.keras.backend.int_shape(x)[2]*tf.keras.backend.int_shape(x)[3]))(x)

    # LSTM 1
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_size, return_sequences=True)
                            )(x)  # [b_s, seq_len, vec_dim]

    # # LSTM 2 TensorShape([None, 51, 800])                                           
    # x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_size, return_sequences=True)
    #                         )(x)  # [b_s, seq_len, vec_dim]

    x = tf.keras.layers.ReLU()(x)
    # Dense 1
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(input_size[1]))(x)
    x = tf.keras.layers.ReLU()(x)
    # Dense 2
    mask = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(input_size[1], activation='sigmoid'))(x)

    output = tf.keras.layers.Multiply()([inputs, mask])
    # print(output.shape)
    model = tf.keras.Model(inputs=[inputs], outputs=[output])
    return model

'learning rate decay'
def step_decay(self, epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 2.0
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate    

### Prepare data

In [6]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
  model = trunk_network()
  model.compile(optimizer='adam', loss=['mse'])
  lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

INFO:tensorflow:Initializing the TPU system: grpc://10.92.214.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.214.90:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [7]:
X_total = []
y_total = []

for i in tqdm(range(67)):
  X_total.append(np.load(f"gdrive/My Drive/Spitter_VTB/mixed_mag/{i}.npy", allow_pickle=True))
  y_total.append(np.load(f"gdrive/My Drive/Spitter_VTB/target_mag/{i}.npy", allow_pickle=True))

100%|██████████| 67/67 [02:03<00:00,  1.85s/it]


In [0]:
model.fit(
  x = X_total, y = y_total,
  steps_per_epoch=67,
  epochs=50,
  callbacks=[lrate]
)

Epoch 1/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0259 - lr: 0.0100
Epoch 2/50
67/67 [==============================] - 9s 131ms/step - loss: 0.0223 - lr: 0.0100
Epoch 3/50
67/67 [==============================] - 9s 131ms/step - loss: 0.0212 - lr: 0.0100
Epoch 4/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0201 - lr: 0.0100
Epoch 5/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0199 - lr: 0.0100
Epoch 6/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0190 - lr: 0.0100
Epoch 7/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0196 - lr: 0.0100
Epoch 8/50
67/67 [==============================] - 9s 131ms/step - loss: 0.0194 - lr: 0.0100
Epoch 9/50
67/67 [==============================] - 9s 130ms/step - loss: 0.0184 - lr: 0.0100
Epoch 10/50
67/67 [==============================] - 9s 131ms/step - loss: 0.0183 - lr: 0.0100
Epoch 11/50
67/67 [==============================] - 9s 130

In [0]:
# with trange(iterations) as t:
#   for iteration in t:
#     i = iteration % total_file_length
    
#     epoch_num = iteration//total_file_length

#     X = X_total[i]
#     y = y_total[i]
#     loss = model.train_on_batch(X, y)
#     t.set_description('epo %i [loss: %f]' % (epoch_num, loss))